# Main analysis script

Put your leakpro results in the *data* folder using the following structure

data/main/*dataset*/*target_model*/.../result.json

Example:

data/main/ELD/LSTM/LiRA-num_shadow_model=64-signal_name=RescaledSMAPELoss.../result.json


In [1]:
from utils import objects_to_ablations
import glob, os, json, gc

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data_folder = "data/eeg_plot"
output_folder = "output/eeg_plot"

# This is the variable that varies and we want to observe the effect of
important_name = "attack"

# This is the parameters given in the structure of the data
from_path = ["dataset", "target_model"]

# The parameters given in config in result.json
from_config = []

config = {
    "ds_indivs": {
        "EEG": 32,
        "ELD": 32,
    },
    "indiv_strategy": "indiv_mean",
    #"order": ["MSELoss", "MAELoss", "SMAPELoss", "RescaledSMAPELoss", "TrendLoss", "SeasonalityLoss", "TS2VecLoss"],
}



In [3]:
objects = []

for path in glob.glob(data_folder + "/**/*.json", recursive=True):
    # Add parameters from path
    data_folder_split = os.path.join(data_folder).split(os.sep)
    path_split = path.split(os.sep)[len(data_folder_split):]
    parameters = {param: path_split[i] for i, param in enumerate(from_path)}

    # Load json file
    with open(path) as f:
        data = json.load(f)

    # Add parameters from config in result.json
    parameters |= {k: v for k,v in data["config"].items() if k in from_config}
    if "signal_names" in data["config"].keys() and "signal_name" in from_config:
        parameters["signal_name"] = ",".join(data["config"]["signal_names"])
    print(data["config"]["online"])

    # Assume attack name to be part of parameters
    parameters |= {"attack": data["result_name"]}
    
    if important_name not in parameters.keys():
        parameters[important_name] = parameters["attack"]

    important_value = parameters[important_name]
    parameters = {k: v for k,v in parameters.items() if k!=important_name}

    objects.append((data, parameters, important_value))
    print((parameters, important_value))


True
({'dataset': 'EEG', 'target_model': 'NHiTS'}, 'DTS-MIA')
True
({'dataset': 'EEG', 'target_model': 'NHiTS'}, 'LiRA')
True
({'dataset': 'EEG', 'target_model': 'NHiTS'}, 'RMIA')


In [4]:
ablations = objects_to_ablations(objects, config)
for study in ablations:
    save_dir = "-".join(f"{k}={v}" for k, v in study.parameters.items())
    save_dir = os.path.join(output_folder, save_dir)
    os.makedirs(save_dir, exist_ok=True) 
    study.make_roc_plot(save_dir)
    study.make_table(save_dir)
    study.make_indiv_roc_plot(save_dir)
    gc.collect() # Garbage collect
    print(f"Saved study to {save_dir}")

Saved study to output/eeg_plot/dataset=EEG-target_model=NHiTS


<Figure size 640x480 with 0 Axes>